In [1]:
!pip install -q pyyaml h5py

In [2]:
import os 

import tensorflow as tf
from tensorflow import keras

In [4]:
# サンプルデータの取得(最初の1000件のみ)
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28*28) / 255.0
test_images = test_images[:1000].reshape(-1, 28*28) / 255.0


In [6]:
# 重みの保存と読み込みのデモを行うための簡単なモデルの定義
# 短いシーケンシャルモデルを返す関数
def create_model():
  model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10, activation='softmax')
  ])

  model.compile(optimizer='adam', 
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

  return model


# 基本的なモデルのインスタンスを作成
model = create_model()

# モデルの構造を表示
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               401920    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [8]:
# 訓練中にチェックポイントを保存する
# 訓練の途中あるいは終了時にチェっっ苦ポイントを自動的に保存
# 再び訓練を行うことなくモデルを使用することができ、訓練が中断された場合中止したところから再開される
checkpoint_path = 'training_1/cp.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)

# チェックポイントコールバックを作る
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                                                        save_weights_only=True,
                                                                                        verbose=1)

# 新しいコールバックを用いるようモデルを訓練
model.fit(train_images,
                 train_labels,
                 epochs=10,
                 validation_data=(test_images, test_labels),
                 callbacks=[cp_callback]) # 訓練にコールバックを渡す

Epoch 1/10
 1/32 [..............................] - ETA: 5s - loss: 2.5319 - accuracy: 0.0312

2022-06-03 13:20:02.078049: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz



Epoch 1: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 6ms/step - loss: 1.1768 - accuracy: 0.6710 - val_loss: 0.7145 - val_accuracy: 0.7870
Epoch 2/10
31/32 [============================>.] - ETA: 0s - loss: 0.4315 - accuracy: 0.8740
Epoch 2: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 4ms/step - loss: 0.4322 - accuracy: 0.8730 - val_loss: 0.5559 - val_accuracy: 0.8230
Epoch 3/10
 1/32 [..............................] - ETA: 0s - loss: 0.2721 - accuracy: 0.9688
Epoch 3: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 4ms/step - loss: 0.2964 - accuracy: 0.9260 - val_loss: 0.4815 - val_accuracy: 0.8560
Epoch 4/10
 1/32 [..............................] - ETA: 0s - loss: 0.2008 - accuracy: 0.9688
Epoch 4: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 3ms/step - loss: 0.2107 - accuracy: 0.9470 - val_loss: 0.4428 - val_accuracy: 0.8590
Epoch 5/10
32/32 [=======

In [9]:
ls {checkpoint_dir}

checkpoint                   cp.ckpt.index
cp.ckpt.data-00000-of-00001


In [11]:
# 訓練していない新しいモデルを作成
model = create_model()

loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 2.3670 - accuracy: 0.1070 - 107ms/epoch - 3ms/step
Untrained model, accuracy: 10.70%


In [12]:
# チェックポイントから重みをロードし、再評価する
model.load_weights(checkpoint_path)
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored mode, accuracy: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.4148 - accuracy: 0.8670 - 34ms/epoch - 1ms/step
Restored mode, accuracy: 86.70%


In [13]:
# チェックポイントコールバックのオプション
# ファイル名に('str.format'を使用して)エポック数を埋め込む
checkpoint_path = 'training_2/cp-{epoch:04d}.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)

# 5エポックごとにモデルの重みを保存するコールバックを作成
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    period=5)

# 新しいモデルのインスタンスを作成
model = create_model()

# 'checkpoint_path' フォーマットで重みを保存
model.save_weights(checkpoint_path.format(epoch=0))

# 新しいコールバックを使い、モデルを訓練
model.fit(train_images,
                 train_labels,
                 epochs=50,
                 callbacks=[cp_callback],
                 validation_data=(test_images, test_labels),
                 verbose=0)


Epoch 5: saving model to training_2/cp-0005.ckpt

Epoch 10: saving model to training_2/cp-0010.ckpt

Epoch 15: saving model to training_2/cp-0015.ckpt

Epoch 20: saving model to training_2/cp-0020.ckpt

Epoch 25: saving model to training_2/cp-0025.ckpt

Epoch 30: saving model to training_2/cp-0030.ckpt

Epoch 35: saving model to training_2/cp-0035.ckpt

Epoch 40: saving model to training_2/cp-0040.ckpt

Epoch 45: saving model to training_2/cp-0045.ckpt

Epoch 50: saving model to training_2/cp-0050.ckpt


In [14]:
ls {checkpoint_dir}

checkpoint                        cp-0025.ckpt.index
cp-0000.ckpt.data-00000-of-00001  cp-0030.ckpt.data-00000-of-00001
cp-0000.ckpt.index                cp-0030.ckpt.index
cp-0005.ckpt.data-00000-of-00001  cp-0035.ckpt.data-00000-of-00001
cp-0005.ckpt.index                cp-0035.ckpt.index
cp-0010.ckpt.data-00000-of-00001  cp-0040.ckpt.data-00000-of-00001
cp-0010.ckpt.index                cp-0040.ckpt.index
cp-0015.ckpt.data-00000-of-00001  cp-0045.ckpt.data-00000-of-00001
cp-0015.ckpt.index                cp-0045.ckpt.index
cp-0020.ckpt.data-00000-of-00001  cp-0050.ckpt.data-00000-of-00001
cp-0020.ckpt.index                cp-0050.ckpt.index
cp-0025.ckpt.data-00000-of-00001


In [15]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2/cp-0050.ckpt'

In [16]:
# 新しいモデルのインスタンスを作成
model = create_model()

# 先ほど保存した重みを読み込み
model.load_weights(latest)

# モデルを再評価
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}".format(100*acc))

32/32 - 0s - loss: 0.4668 - accuracy: 0.8740 - 106ms/epoch - 3ms/step
Restored model, accuracy: 87.40


In [19]:
# 手動で重みを保存する
# 重みの保存
model.save_weights('./checkpoints/my_checkpoint')

model = create_model()
model.load_weights('./checkpoints/my_checkpoint')
loss,acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.4668 - accuracy: 0.8740 - 104ms/epoch - 3ms/step
Restored model, accuracy: 87.40%


In [23]:
# モデル全体をSavedModelフォーマットとして保存
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# モデル全体を SavedModel として保存
!mkdir -p saved_model
model.save('saved_model/my_model')

Epoch 1/5
32/32 [==============================] - 0s 1ms/step - loss: 1.1415 - accuracy: 0.6690
Epoch 2/5
32/32 [==============================] - 0s 2ms/step - loss: 0.4299 - accuracy: 0.8830
Epoch 3/5
32/32 [==============================] - 0s 2ms/step - loss: 0.2921 - accuracy: 0.9160
Epoch 4/5
32/32 [==============================] - 0s 2ms/step - loss: 0.2048 - accuracy: 0.9590
Epoch 5/5
32/32 [==============================] - 0s 2ms/step - loss: 0.1620 - accuracy: 0.9630
INFO:tensorflow:Assets written to: saved_model/my_model/assets


In [26]:
# my_model ディレクトリ
!ls saved_model

# assetsフォルダとsaved_model.pb, variablesフォルダが含まれる
!ls saved_model/my_model

my_model
assets            keras_metadata.pb saved_model.pb    variables


In [27]:
new_model = tf.keras.models.load_model('saved_model/my_model')

# アーキテクチャを確認
new_model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_14 (Dense)            (None, 512)               401920    
                                                                 
 dropout_7 (Dropout)         (None, 512)               0         
                                                                 
 dense_15 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [28]:
# リストアされたモデルを評価
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100*acc))

print(new_model.predict(test_images).shape)

32/32 - 0s - loss: 0.4334 - accuracy: 0.8570 - 85ms/epoch - 3ms/step
Restored model, accuracy: 85.70%
(1000, 10)


In [29]:
# HDF5ファイルとしてモデル全体を保存
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# HDF5 ファイルにモデル全体を保存
# 拡張子 '.h5' はモデルが HDF5 で保存されているということを暗示する
model.save('my_model.h5')

Epoch 1/5
32/32 [==============================] - 0s 1ms/step - loss: 1.1009 - accuracy: 0.6930
Epoch 2/5
32/32 [==============================] - 0s 1ms/step - loss: 0.4210 - accuracy: 0.8750
Epoch 3/5
32/32 [==============================] - 0s 2ms/step - loss: 0.2800 - accuracy: 0.9320
Epoch 4/5
32/32 [==============================] - 0s 1ms/step - loss: 0.2044 - accuracy: 0.9480
Epoch 5/5
32/32 [==============================] - 0s 2ms/step - loss: 0.1445 - accuracy: 0.9700


In [30]:
# 同じモデルを読み込んで、重みやオプティマイザーを含むモデル全体を再作成
new_model = tf.keras.models.load_model('my_model.h5')

# モデルのアーキテクチャを表示
new_model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_16 (Dense)            (None, 512)               401920    
                                                                 
 dropout_8 (Dropout)         (None, 512)               0         
                                                                 
 dense_17 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [31]:
loss, acc = new_model.evaluate(test_images,  test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.4298 - accuracy: 0.8590 - 82ms/epoch - 3ms/step
Restored model, accuracy: 85.90%
